In [6]:
import yfinance as yf
import numpy as np

class DCF:
    def __init__(self, ticker):
        self.ticker = ticker
        self.stock = yf.Ticker(ticker)

    def calculate_fcf(self):
        cashflow = self.stock.cashflow
        if 'Free Cash Flow' in cashflow.index:
            fcf = cashflow.loc['Free Cash Flow']
        else:
            print("Free Cash Flow data is not available.")
            fcf = None
        return fcf
    
    def calculate_wacc(self):
        balance_sheet = self.stock.balance_sheet
        income_statement = self.stock.financials
        ebit=income_statement.loc['EBIT']
        interest_expense = income_statement.loc['Interest Expense']
        tax_rate = 0.125
        risk_free_rate = 0.02
        market_preimum = 0.05
        beta=1.2
        cost_of_equity=risk_free_rate+beta*market_preimum
        cost_of_debt=(interest_expense)*(1-tax_rate)
        wacc=(cost_of_equity+cost_of_debt)*(1+tax_rate)
        return wacc.dropna()
    
    def terminal_value(self):
        fcf = self.calculate_fcf()
        wacc = self.calculate_wacc()
        growth_rate=0.04
        terminal_value=fcf*(1+growth_rate)/(wacc-growth_rate)
        terminal_value=terminal_value.dropna()
        return terminal_value[::-1]
    
    def calculate_dcf(self):
        fcf = self.calculate_fcf()
        wacc = self.calculate_wacc()
        terminal_value = self.terminal_value()
        dcf=0
        for i in range(0,len(fcf)):
            dcf+=fcf[i]/(1+wacc)**(i+1)
        dcf+=terminal_value/(1+wacc)**len(fcf)
        return dcf.dropna()
    
    
    def get_outstanding_shares(self):
        try:
            shares = self.stock.info['sharesOutstanding']
        except KeyError as e:
            print(f"KeyError: {e}")
            shares = None
        return shares
    
    def calculate_projected_share_price(self, dcf_value):
        shares_outstanding = self.get_outstanding_shares()
        if dcf_value is not None and shares_outstanding is not None:
            projected_share_price = dcf_value / shares_outstanding
        else:
            print("Cannot calculate projected share price due to missing data.")
            projected_share_price = None
        return projected_share_price

# Example usage
dcf_model = DCF('AAPL')  # Replace 'AAPL' with the desired ticker symbol
dcf_value = dcf_model.calculate_dcf()


/var/folders/mk/fn7b709x3dz4m442ngj1_xf00000gn/T/ipykernel_7172/284051278.py:46: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dcf+=fcf[i]/(1+wacc)**(i+1)
